# DSE511 • Fairness Audit Mini‑Project (Starter Notebook)

**Goal:** Train a baseline classifier on the synthetic admissions dataset, compute group fairness metrics, explore interpretability (SHAP), and try one mitigation. Keep code clean and reproducible.

**Deliverables:** This filled notebook + a short PDF write‑up (≤ 2 pages) with optional figures and interpretation.

**Dataset:** `admissions_synth_v1.csv` (bias‑infused, synthetic). Data dictionary provided separately.


## 0) Setup & Reproducibility

- Use fixed random seeds.
- Avoid relying on global state.
- Keep cells idempotent.
- Python ≥ 3.9 recommended.


In [ ]:
import numpy as np, pandas as pd
from pathlib import Path
RANDOM_STATE = 7
np.random.seed(RANDOM_STATE)
DATA_PATH = Path('admissions_synth_v1.csv')
assert DATA_PATH.exists(), 'Place admissions_synth_v1.csv in this directory.'
pd.__version__

## 1) Load Data & Quick EDA
Use **only** matplotlib for plots in this notebook (no seaborn), per style guidance.

In [ ]:
import matplotlib.pyplot as plt
df = pd.read_csv(DATA_PATH)
df.head()

In [ ]:
# Class balance
admit_rate = df['admit'].mean()
print('Overall admit rate:', round(admit_rate, 3))

# Simple bar chart of admit rate by race
by_race = df.groupby('race')['admit'].mean().sort_values()
plt.figure()
by_race.plot(kind='bar')
plt.xlabel('Race')
plt.ylabel('Admit rate')
plt.title('Admit rate by race')
plt.show()

## 2) Train/Test Split & Baseline Model
Baseline: Logistic Regression (you may also try RandomForest or GradientBoosting later).

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix

y = df['admit'].astype(int).values
X = df.drop(columns=['admit'])

num_cols = ['gpa','sat_total','ap_courses','rec_score','interview_score',
            'ecs_hours_per_week','income_quintile','in_state','legacy','first_gen']
cat_cols = ['gender','race','intended_major']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=RANDOM_STATE
)

pre = ColumnTransformer([
    ('num', StandardScaler(), num_cols),
    ('cat', OneHotEncoder(drop='first', handle_unknown='ignore'), cat_cols)
])

clf = Pipeline([
    ('prep', pre),
    ('lr', LogisticRegression(max_iter=1000, random_state=RANDOM_STATE))
])

clf.fit(X_train, y_train)
proba = clf.predict_proba(X_test)[:,1]
pred = (proba >= 0.5).astype(int)

print('ROC-AUC:', round(roc_auc_score(y_test, proba), 3))
print('Accuracy:', round(accuracy_score(y_test, pred), 3))
print('Confusion matrix @0.5:\n', confusion_matrix(y_test, pred))

## 3) Group Fairness Metrics
Compute **Demographic Parity (selection rate)**, **True Positive Rate (TPR)**, and **False Positive Rate (FPR)** by group, then summarize gaps.

In [ ]:
import numpy as np
import pandas as pd

def group_metrics(y_true, y_pred, groups):
    out = {}
    for g in np.unique(groups):
        mask = (groups == g)
        yt, yp = y_true[mask], y_pred[mask]
        sel = yp.mean()
        tp = ((yp==1) & (yt==1)).sum()
        fn = ((yp==0) & (yt==1)).sum()
        fp = ((yp==1) & (yt==0)).sum()
        tn = ((yp==0) & (yt==0)).sum()
        tpr = tp/(tp+fn) if (tp+fn)>0 else np.nan
        fpr = fp/(fp+tn) if (fp+tn)>0 else np.nan
        out[g] = {'selection_rate': sel, 'TPR': tpr, 'FPR': fpr, 'n': mask.sum()}
    return pd.DataFrame(out).T

# Example: race and first_gen
race_test = X_test['race'].values
gm_race = group_metrics(y_test, pred, race_test)
display(gm_race)

fg_test = X_test['first_gen'].values
gm_fg = group_metrics(y_test, pred, fg_test)
display(gm_fg)

# Gap summaries
def summarize_gaps(gdf):
    return pd.Series({
        'DP_diff (max-min selection)': gdf['selection_rate'].max() - gdf['selection_rate'].min(),
        'EO_diff (max-min TPR)': gdf['TPR'].max() - gdf['TPR'].min(),
        'AvgOdds_diff (avg of FPR/TPR gaps)': 0.5*((gdf['TPR'].max()-gdf['TPR'].min()) + (gdf['FPR'].max()-gdf['FPR'].min()))
    })

print('Race gaps:')
display(summarize_gaps(gm_race))
print('First‑gen gaps:')
display(summarize_gaps(gm_fg))

## 4) Threshold Exploration (Post‑processing idea)
Explore group‑specific thresholds to equalize TPR or selection rates, then re‑report accuracy and fairness.

In [ ]:
def threshold_by_group(proba, base_thresh, groups, policy='equal_tpr'):
    # Simple illustrative approach: search thresholds per group on validation-like split
    # Here, we just nudge thresholds by z-score of group mean proba as a toy method.
    th = {}
    unique = np.unique(groups)
    for g in unique:
        m = proba[groups==g].mean()
        s = proba.mean()
        # if group probas are lower than overall, reduce threshold a bit, else increase
        th[g] = float(base_thresh - 0.1*(s - m))
    return th

base = 0.5
race_groups = X_test['race'].values
tmap = threshold_by_group(proba, base, race_groups)
pred_gp = np.array([1 if p>=tmap[g] else 0 for p,g in zip(proba, race_groups)])

print('Accuracy (group thresholds):', round(accuracy_score(y_test, pred_gp), 3))
gm_race_adj = group_metrics(y_test, pred_gp, race_groups)
display(gm_race_adj)
display(summarize_gaps(gm_race_adj))

## 5) Interpretability (SHAP)
Use Kernel SHAP over the pipeline prediction function. Keep background small for speed.

In [ ]:
import shap

# Wrapper to pass raw DataFrame to the fitted pipeline
def f_predict(Xraw):
    # Ensure DataFrame with same columns/order as training
    if not isinstance(Xraw, pd.DataFrame):
        Xraw = pd.DataFrame(Xraw, columns=X_test.columns)
    return clf.predict_proba(Xraw)[:,1]

bg = X_train.sample(200, random_state=RANDOM_STATE)
explainer = shap.KernelExplainer(f_predict, bg)
X_eval = X_test.sample(200, random_state=RANDOM_STATE)
shap_vals = explainer.shap_values(X_eval)

shap.summary_plot(shap_vals, X_eval, feature_names=X_eval.columns)

## 6) Mitigation (Choose ONE)

Try **one** of the options below and re‑audit fairness & utility:

1. **Pre‑processing (reweighing / resampling):** upsample positives for under‑selected groups.
2. **In‑processing (approximate):** add group‑aware sample weights to the loss (e.g., weight first‑gen or specific race groups).
3. **Post‑processing:** adopt group‑specific thresholds to equalize TPR or selection rate.

Document: What changed? What trade‑offs occurred?

In [ ]:
# Example: simple pre‑processing via group‑aware upsampling (toy illustration)
from sklearn.utils import resample

train = X_train.copy()
train['admit'] = y_train
minority_mask = (train['first_gen']==1)
maj = train[~minority_mask]
minr = train[minority_mask]

# Upsample first‑gen positives to reduce selection/TPR gaps (toy; tune as needed)
minr_pos = minr[minr['admit']==1]
if len(minr_pos) > 0:
    minr_pos_up = resample(minr_pos, replace=True, n_samples=min(len(maj), len(minr_pos)*2), random_state=RANDOM_STATE)
    train_up = pd.concat([maj, minr, minr_pos_up], ignore_index=True)
else:
    train_up = train.copy()

y_train_up = train_up['admit'].astype(int).values
X_train_up = train_up.drop(columns=['admit'])

clf_up = Pipeline([
    ('prep', pre),
    ('lr', LogisticRegression(max_iter=1000, random_state=RANDOM_STATE))
])

clf_up.fit(X_train_up, y_train_up)
proba_up = clf_up.predict_proba(X_test)[:,1]
pred_up = (proba_up >= 0.5).astype(int)

print('Mitigated ROC-AUC:', round(roc_auc_score(y_test, proba_up), 3))
print('Mitigated Accuracy:', round(accuracy_score(y_test, pred_up), 3))
gm_fg_up = group_metrics(y_test, pred_up, X_test['first_gen'].values)
display(gm_fg_up)
display(summarize_gaps(gm_fg_up))

## 7) Reflection (200–300 words)

- Which **fairness definition(s)** did you prioritize and why?
- Is your model **trustworthy** for this use case? Under which constraints or policies?
- What risks or unintended harms remain after mitigation?


---
### Appendix: Notes
- You may also try `RandomForestClassifier` or `HistGradientBoostingClassifier`.
- For calibration by group, consider reliability curves (matplotlib only).
- Keep the same **test split** across experiments for comparability if you can.
